Atreish Ramlakhan<br> Aishwarya Singh<br> Nosson Weissman<br>
April 22, 2021 
AIM 5001, Prof. James Topor  <br>

# AIM 5001 Module 13
## PostgreSQL to MongoDB Data Migration


*The task for the Module 13 Assignment is to migrate the content of a PostgreSQL database to MongoDB via 
Python, and then retrieve the data from MongoDB and load it into Pandas dataframes that conform to the 
format of the original PostgreSQL database tables. Specifically, you will use Python to retrieve the entire 
content of the Customer and Invoice tables from the Chinook database we’ve worked within in Modules 1 & 2 
from your PostgreSQL server, and then subsequently load that data into MongoDB. The manner in which you 
choose to store and manage the Chinook data within MongoDB is up to you as an SQL and NoSQL practitioner 
to decide. 
Then, using Python and your MongoDB data retrieval expertise, you will retrieve the required Chinook data 
from MongoDB and create and populate a pair of new Pandas dataframes, one for each of the original 
Chinook Customer and Invoice database tables. When finished with this process, your new Pandas dataframes 
should mirror the structure of the SQL tables contained within your PostgreSQL Chinook database.*

1) Introduction (5 Points): Summarize the problem + explain the steps you plan to take to address the problem. <br> 
**Our goal is to migrate some tables from a Postgres database to a Mongo database while maintaining a similar structure.<br>
We shall start by reviewing the structure of the orginal tables we intend to migrate.<br>
We will use pandas to convert the tables to dataframes and ensure dataypes are valid types for a Mongo database.**

2) PostgreSQL to MongoDB Migration (55 Points): Describe + show the steps you have taken to use 
Python to retrieve the Chinook data from your PostgreSQL server and subsequently load that data into 
MongoDB. This section should include any Python code used as well as an appropriate explanatory 
narrative that explains how you have formatted the Chinook data for storage within MongoDB.

In [41]:
# Install pymongo
# !pip install pymongo

In [42]:
#first import libraries
import pymongo
from pymongo import MongoClient
import psycopg2 as pg
import pandas as pd, numpy as np

### Access SQL Database, Import Tables

In [47]:
import os
#here we retirive the databse password which is stored as an environment variable
#best practice not to save passwords as plaintext in notebooks
#(we can also create env variables for port, database, user, and sslmode)
password = os.getenv('POSTGRES_PASSWORD')

In [43]:
conn = pg.connect(
    host="localhost",
    port="5433",
    database= database_name,
    user="postgres",
    password=password,
    sslmode="disable")
query = """SELECT * FROM "Customer" """ 
customer = pd.read_sql_query(query, conn)
query = """SELECT * FROM "Invoice" """ 
invoice = pd.read_sql_query(query, conn)

### Access Mongo Server, Create New Database

In [44]:
client = MongoClient('localhost', 27017)
# client = MongoClient('mongodb://localhost:27017/')
db = client.test_database
# db = client['test-database']

In [45]:
database_name = 'chinook'
db = client[database_name]

In [46]:
client.list_database_names()

['admin', 'chinook', 'config', 'local', 'test_database']

In [48]:
# table_df = customer
# documents = []
# for i in table_df.index:
#      documents.append(customer.iloc[i].to_dict())
# collection = customers
# collection.insert_many(documents)
# customers
# cols = customer.columns
# first_row = customer.iloc[0]
# for col in cols:
#     if type(first_row[col]) == np.int64:
#         print(type(np.long(first_row[col])))
##InvalidDocument: cannot encode object: 1, of type: <class 'numpy.int64'>

In [49]:
#create a function to convert dataframe to collection
def table_df_to_mongo_collection(table_df):
    docs = []
    df_cols = [s for s in table_df.columns]
    for i in table_df.index:
        doc = {}
        for col in df_cols:
            #Mongo does not recognize np.int64
            #here we convert to n.long which is a valid type for Mongo
            #ideally, we should create a dict with numpy types 
            #which aren't recogniable in Mongo and their Mongo equivalent
            if type(table_df.iloc[i][col]) == np.int64:
                doc[col] = np.long(table_df.iloc[i][col])
            else:
                doc[col] = table_df.iloc[i][col]
        docs.append(doc)
    return docs

In [50]:
def convert_and_insert_df_as_collection(db, table_df, new_collection_name):
    if not (db and table_df and new_collection_name):
        raise Exception('''One of the parameters you've entered are of type None''')
    collection = db[new_collection_name]
    docs = table_df_to_mongo_collection(table_df)
    collection.insert_many(docs)
    
customers = db.invoices
customers.drop()
invoices = db.invoices
invoices.drop()
convert_and_insert_df_as_collection(db, customer, 'customers')
convert_and_insert_df_as_collection(db, invoice, 'invoices')
# [s for s in invoice.columns]

In [84]:
print('heyyy \U0001F923 \N{winking face}\N{snake}')

heyyy 🤣 😉🐍


In [51]:
client.list_database_names()

['admin', 'chinook', 'config', 'local', 'test_database']

In [52]:
db = client.chinook

In [53]:
db.list_collection_names()

['invoices', 'customers']

In [54]:
# customers = db.invoices
# customers.drop()
[s for s in db['invoices'].find()]

[{'_id': ObjectId('6081e178d46ebf23e5ecf24b'),
  'InvoiceId': 1,
  'CustomerId': 2,
  'InvoiceDate': datetime.datetime(2009, 1, 1, 0, 0),
  'BillingAddress': 'Theodor-Heuss-Straße 34',
  'BillingCity': 'Stuttgart',
  'BillingState': None,
  'BillingCountry': 'Germany',
  'BillingPostalCode': '70174',
  'Total': 1.98},
 {'_id': ObjectId('6081e178d46ebf23e5ecf24c'),
  'InvoiceId': 2,
  'CustomerId': 4,
  'InvoiceDate': datetime.datetime(2009, 1, 2, 0, 0),
  'BillingAddress': 'Ullevålsveien 14',
  'BillingCity': 'Oslo',
  'BillingState': None,
  'BillingCountry': 'Norway',
  'BillingPostalCode': '0171',
  'Total': 3.96},
 {'_id': ObjectId('6081e178d46ebf23e5ecf24d'),
  'InvoiceId': 3,
  'CustomerId': 8,
  'InvoiceDate': datetime.datetime(2009, 1, 3, 0, 0),
  'BillingAddress': 'Grétrystraat 63',
  'BillingCity': 'Brussels',
  'BillingState': None,
  'BillingCountry': 'Belgium',
  'BillingPostalCode': '1000',
  'Total': 5.94},
 {'_id': ObjectId('6081e178d46ebf23e5ecf24e'),
  'InvoiceId': 4,

3) MongoDB to Pandas Migration (40 Points): Describe + show the steps you have taken in Python to 
retrieve the Chinook data from MongoDB and create + populate the required Pandas dataframes that 
mirror the structure of the SQL tables contained within your PostgreSQL Chinook database. This 
section should include any Python code used for Data Preparation as well as an appropriate 
explanatory narrative.
Your Jupyter Notebook deliverable should be similar to that of a publication-quality / professional caliber 
document and should include clearly labeled graphics, high-quality formatting, clearly defined section and 
sub-section headers, and be free of spelling and grammar errors. Furthermore, your Pythion code should 
include succinct explanatory comments. 
Upload / submit your Jupyter Notebook within the provided M13 Assignment Canvas submission portal. Be 
sure to save your Notebook using the following nomenclature: first initial_last name_M13_assn" (e.g., 
J_Smith_M13_assn). Small groups should identity all group members at the start of the Jupyter Notebook 
and each team member should submit their own copy of the team’s work within Canvas

In [55]:
for customer in customers.find():
    print(customer)

{'_id': ObjectId('6081e178d46ebf23e5ecf24b'), 'InvoiceId': 1, 'CustomerId': 2, 'InvoiceDate': datetime.datetime(2009, 1, 1, 0, 0), 'BillingAddress': 'Theodor-Heuss-Straße 34', 'BillingCity': 'Stuttgart', 'BillingState': None, 'BillingCountry': 'Germany', 'BillingPostalCode': '70174', 'Total': 1.98}
{'_id': ObjectId('6081e178d46ebf23e5ecf24c'), 'InvoiceId': 2, 'CustomerId': 4, 'InvoiceDate': datetime.datetime(2009, 1, 2, 0, 0), 'BillingAddress': 'Ullevålsveien 14', 'BillingCity': 'Oslo', 'BillingState': None, 'BillingCountry': 'Norway', 'BillingPostalCode': '0171', 'Total': 3.96}
{'_id': ObjectId('6081e178d46ebf23e5ecf24d'), 'InvoiceId': 3, 'CustomerId': 8, 'InvoiceDate': datetime.datetime(2009, 1, 3, 0, 0), 'BillingAddress': 'Grétrystraat 63', 'BillingCity': 'Brussels', 'BillingState': None, 'BillingCountry': 'Belgium', 'BillingPostalCode': '1000', 'Total': 5.94}
{'_id': ObjectId('6081e178d46ebf23e5ecf24e'), 'InvoiceId': 4, 'CustomerId': 14, 'InvoiceDate': datetime.datetime(2009, 1, 6,

In [56]:
# customers = db.invoices
# customers.drop()
[s for s in db['invoices'].find()]

[{'_id': ObjectId('6081e178d46ebf23e5ecf24b'),
  'InvoiceId': 1,
  'CustomerId': 2,
  'InvoiceDate': datetime.datetime(2009, 1, 1, 0, 0),
  'BillingAddress': 'Theodor-Heuss-Straße 34',
  'BillingCity': 'Stuttgart',
  'BillingState': None,
  'BillingCountry': 'Germany',
  'BillingPostalCode': '70174',
  'Total': 1.98},
 {'_id': ObjectId('6081e178d46ebf23e5ecf24c'),
  'InvoiceId': 2,
  'CustomerId': 4,
  'InvoiceDate': datetime.datetime(2009, 1, 2, 0, 0),
  'BillingAddress': 'Ullevålsveien 14',
  'BillingCity': 'Oslo',
  'BillingState': None,
  'BillingCountry': 'Norway',
  'BillingPostalCode': '0171',
  'Total': 3.96},
 {'_id': ObjectId('6081e178d46ebf23e5ecf24d'),
  'InvoiceId': 3,
  'CustomerId': 8,
  'InvoiceDate': datetime.datetime(2009, 1, 3, 0, 0),
  'BillingAddress': 'Grétrystraat 63',
  'BillingCity': 'Brussels',
  'BillingState': None,
  'BillingCountry': 'Belgium',
  'BillingPostalCode': '1000',
  'Total': 5.94},
 {'_id': ObjectId('6081e178d46ebf23e5ecf24e'),
  'InvoiceId': 4,

In [57]:
db.list_collection_names()

['invoices', 'customers']

In [58]:
db = client.chinook

In [59]:
# Install pymongo
# !pip install pymongo

In [60]:
#first import libraries
import pymongo
from pymongo import MongoClient
import psycopg2 as pg
import pandas as pd, numpy as np

In [61]:
client = MongoClient('localhost', 27017)
# client = MongoClient('mongodb://localhost:27017/')

In [62]:
db = client.test_database
# db = client['test-database']

In [63]:
database_name = 'chinook'
db = client[database_name]

In [64]:
import os
#here we retirive the databse password which is stored as an environment variable
#best practice not to save passwords as plaintext in notebooks
#(we can also create env variables for port, database, user, and sslmode)
password = os.getenv('POSTGRES_PASSWORD')

In [65]:
conn = pg.connect(
    host="localhost",
    port="5433",
    database= database_name,
    user="postgres",
    password=password,
    sslmode="disable")
query = """SELECT * FROM "Customer" """ 
customer = pd.read_sql_query(query, conn)
query = """SELECT * FROM "Invoice" """ 
invoice = pd.read_sql_query(query, conn)

In [66]:
# table_df = customer
# documents = []
# for i in table_df.index:
#      documents.append(customer.iloc[i].to_dict())
# collection = customers
# collection.insert_many(documents)
# customers
cols
first_row = customer.iloc[0]
for col in cols:
    if type(first_row[col]) == np.int64:
        print(type(np.long(first_row[col])))
##InvalidDocument: cannot encode object: 1, of type: <class 'numpy.int64'>

<class 'int'>
<class 'int'>


In [67]:
def df_to_mongo_collection(table_df):
    docs = []
    df_cols = [s for s in table_df.columns]
    for i in table_df.index:
        doc = {}
        for col in df_cols:
            #Mongo does not recognize np.int64
            #here we convert to n.long which is a valid type for Mongo
            #ideally, we should create a dict with numpy types 
            #which aren't recogniable in Mongo and their Mongo equivalent
            if type(table_df.iloc[i][col]) == np.int64:
                doc[col] = np.long(table_df.iloc[i][col])
            else:
                doc[col] = table_df.iloc[i][col]
        docs.append(doc)
    return docs

In [68]:
def convert_and_insert_df_as_collection(db, table_df, new_collection_name):
    if not db and table_df and new_collection_name:
        raise Exception('''One of the parameters you've entered are of type None''')
    collection = db[new_collection_name]
    docs = df_to_mongo_collection(table_df)
    collection.insert_many(docs)
    
customers = db.invoices
customers.drop()
invoices = db.invoices
invoices.drop()
convert_and_insert_df_as_collection(db, customer, 'customers')
convert_and_insert_df_as_collection(db, invoice, 'invoices')
# [s for s in invoice.columns]

In [69]:
len([s for s in db['invoices'].find()])
print([s for s in db['invoices'].find()])

[{'_id': ObjectId('6081e17ad46ebf23e5ecf423'), 'InvoiceId': 1, 'CustomerId': 2, 'InvoiceDate': datetime.datetime(2009, 1, 1, 0, 0), 'BillingAddress': 'Theodor-Heuss-Straße 34', 'BillingCity': 'Stuttgart', 'BillingState': None, 'BillingCountry': 'Germany', 'BillingPostalCode': '70174', 'Total': 1.98}, {'_id': ObjectId('6081e17ad46ebf23e5ecf424'), 'InvoiceId': 2, 'CustomerId': 4, 'InvoiceDate': datetime.datetime(2009, 1, 2, 0, 0), 'BillingAddress': 'Ullevålsveien 14', 'BillingCity': 'Oslo', 'BillingState': None, 'BillingCountry': 'Norway', 'BillingPostalCode': '0171', 'Total': 3.96}, {'_id': ObjectId('6081e17ad46ebf23e5ecf425'), 'InvoiceId': 3, 'CustomerId': 8, 'InvoiceDate': datetime.datetime(2009, 1, 3, 0, 0), 'BillingAddress': 'Grétrystraat 63', 'BillingCity': 'Brussels', 'BillingState': None, 'BillingCountry': 'Belgium', 'BillingPostalCode': '1000', 'Total': 5.94}, {'_id': ObjectId('6081e17ad46ebf23e5ecf426'), 'InvoiceId': 4, 'CustomerId': 14, 'InvoiceDate': datetime.datetime(2009, 1